In [36]:
import pandas, sqlalchemy
from datetime import datetime
import pandas as pd
import math

In [2]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

In [3]:
Base = automap_base()

In [4]:
engine = create_engine('postgres+psycopg2://postgres@localhost:5436/anherf')

/Users/likit/.virtualenvs/anherf/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [5]:
Base.prepare(engine, reflect=True)

In [6]:
Base.classes.keys()

['alembic_version', 'registrations', 'participants', 'roles']

In [7]:
session = Session(engine)

In [15]:
Role = Base.classes.roles

In [13]:
Participant = Base.classes['participants']

In [14]:
Registration = Base.classes['registrations']

In [82]:
def add_new_participant(role, data):
    for item in data.iterrows():
        row = item[1]
        existing_par = session.query(Participant).filter(Participant.id==row['ID']).first()
        if existing_par:
            continue
        payment_required = True if row['payment_required']==1.0 else False
        pay_status = False if pd.isna(row['paid_on']) else True
        payment_date = None if pd.isna(row['paid_on']) else row['paid_on']
        new_par = Participant(old_id=row['ID'],
                        title=row['name_title'],
                         firstname=row['first_name'],
                         lastname=row['last_name'],
                         email=row['user_email'],
                         faculty=row['faculty'],
                         affil=row['university'],
                         mobile=row['mobile'],
                         delivery_address=row['delivery_address'],
                         position_type=row['position_type'],
                         role_id=role.id)
        registered_at = row['user_registered']
        new_par.registrations_collection.append(Registration(registered_at=registered_at,
                                                    pay_status=pay_status,
                                                    payment_required=payment_required,
                                                    paid_on=payment_date))
        session.add(new_par)
    session.commit()

In [30]:
committee_data = pandas.read_excel('register 04.10.2018.xlsx', sheet_name='committee')

In [31]:
speaker_data = pandas.read_excel('register 04.10.2018.xlsx', sheet_name='speaker')

In [33]:
speaker_role = session.query(Roles).filter(Roles.desc=='speaker').first()
add_new_participant(speaker_role, speaker_data)

In [35]:
committee_role = session.query(Roles).filter(Roles.desc=='committee').first()
add_new_participant(committee_role, committee_data)

In [84]:
participant_data = pandas.read_excel('register 09.10.2018.xlsx', sheet_name='participant')

In [85]:
participant_role = session.query(Roles).filter(Roles.desc=='participant').first()
add_new_participant(participant_role, participant_data)

In [83]:
session.rollback()

In [66]:
participant_data[['payment_required', 'payment_date']]

,payment_required,payment_date
0,True,NaN
1,True,NaN
2,True,NaN
3,True,NaN
4,True,NaN
5,True,NaN
6,True,NaN
7,True,NaN
8,True,NaN
9,True,NaN
